<a href="https://colab.research.google.com/github/Praxis-QR/RDWH/blob/main/SQL_for_Data_Science_L3_Procedures_%26_K_Means.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


![alt text](https://4.bp.blogspot.com/-gbL5nZDkpFQ/XScFYwoTEII/AAAAAAAAAGY/CcVb_HDLwvs2Brv5T4vSsUcz7O4r2Q79ACK4BGAYYCw/s1600/kk3-header00-beta.png)<br>


<hr>

[Prithwis Mukerjee](http://www.linkedin.com/in/prithwis)<br>

#K-Means Clustering with SQL 
<br>

Based on the code given by [Joni Salonen](http://jonisalonen.com/2012/k-means-clustering-in-mysql/)

# Install MySQL

In [1]:
# install mysql on colab
#!apt-get -y install mysql-server
!apt -y install mysql-server

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'apt autoremove' to remove it.
The following additional packages will be installed:
  libcgi-fast-perl libcgi-pm-perl libencode-locale-perl libevent-core-2.1-6
  libfcgi-perl libhtml-parser-perl libhtml-tagset-perl libhtml-template-perl
  libhttp-date-perl libhttp-message-perl libio-html-perl
  liblwp-mediatypes-perl libtimedate-perl liburi-perl mysql-client-5.7
  mysql-client-core-5.7 mysql-server-5.7 mysql-server-core-5.7
Suggested packages:
  libdata-dump-perl libipc-sharedcache-perl libwww-perl mailx tinyca
The following NEW packages will be installed:
  libcgi-fast-perl libcgi-pm-perl libencode-locale-perl libevent-core-2.1-6
  libfcgi-perl libhtml-parser-perl libhtml-tagset-perl libhtml-template-perl
  libhttp-date-perl libhttp-message-perl libio-html-perl
  liblwp-mediatypes-perl li

In [2]:
!ls /etc/init.d/mysql

/etc/init.d/mysql


In [3]:
!/etc/init.d/mysql restart

 * Stopping MySQL database server mysqld
   ...done.
 * Starting MySQL database server mysqld
No directory, logging in with HOME=/
   ...done.


In [10]:
!mysql -e 'show databases'

+--------------------+
| Database           |
+--------------------+
| information_schema |
| mysql              |
| performance_schema |
| praxisDB           |
| sys                |
+--------------------+


In [7]:
!mysql -e 'create database praxisDB'
!mysql -e 'show databases'

ERROR 1007 (HY000) at line 1: Can't create database 'praxisDB'; database exists
+--------------------+
| Database           |
+--------------------+
| information_schema |
| mysql              |
| performance_schema |
| praxisDB           |
| sys                |
+--------------------+


# Basic K-Means <br>
the code is available on [this blog post](http://blog.yantrajaal.com/2016/11/k-means-clustering-with-sql.html).

In [17]:
!mysql praxisDB -e 'drop table if exists km_data;'  
!mysql praxisDB -e 'drop table if exists km_clusters;'  
!mysql praxisDB -e 'drop table if exists km_steps;'    

In [18]:
!mysql praxisDB -e 'create table km_data (id int primary key, cluster_id int, x1 float, x2 float);'
!mysql praxisDB -e 'create table km_clusters (id int auto_increment primary key, x1 float, x2 float);'
!mysql praxisDB -e 'create table km_steps (id int, x1 float, x2 float);'

In [20]:
%%writefile SQLproc.sql
drop procedure if exists kmeans; 
DELIMITER //                    
CREATE PROCEDURE kmeans(v_K int)
BEGIN                           
TRUNCATE km_clusters;          
truncate km_steps;              

-- initialize cluster centers 
INSERT INTO km_clusters (x1, x2) SELECT x1, x2 FROM km_data LIMIT v_K; \
REPEAT 
    insert into km_steps select * from km_clusters; 
    -- assign clusters to data points 
    UPDATE km_data d SET cluster_id = (SELECT id FROM km_clusters c 
        ORDER BY POW(d.x1-c.x1,2)+POW(d.x2-c.x2,2) ASC LIMIT 1); 
    -- calculate new cluster center
    UPDATE km_clusters C, (SELECT cluster_id, 
        AVG(x1) AS x1, AVG(x2) AS x2 
        FROM km_data GROUP BY cluster_id) D 
  SET C.x1=D.x1, C.x2=D.x2 WHERE C.id=D.cluster_id; 
UNTIL ROW_COUNT() = 0 END REPEAT; 
END//

Writing SQLproc.sql


In [21]:
!mysql praxisDB --table < SQLproc.sql

In [22]:
!mysql praxisDB -e 'truncate km_data;'
!mysql praxisDB -e 'insert into km_data (id, cluster_id,x1,x2) values (1,0,185,72);'
!mysql praxisDB -e 'insert into km_data (id, cluster_id,x1,x2) values (2,0,170,56);'
!mysql praxisDB -e 'insert into km_data (id, cluster_id,x1,x2) values (3,0,168,60);'
!mysql praxisDB -e 'insert into km_data (id, cluster_id,x1,x2) values (4,0,179,68);'
!mysql praxisDB -e 'insert into km_data (id, cluster_id,x1,x2) values (5,0,182,72);'
!mysql praxisDB -e 'insert into km_data (id, cluster_id,x1,x2) values (6,0,188,77);'

!mysql praxisDB -e 'insert into km_data (id, cluster_id,x1,x2) values (7,0,180,71);'
!mysql praxisDB -e 'insert into km_data (id, cluster_id,x1,x2) values (8,0,180,70);'
!mysql praxisDB -e 'insert into km_data (id, cluster_id,x1,x2) values (9,0,183,84);'
!mysql praxisDB -e 'insert into km_data (id, cluster_id,x1,x2) values (10,0,180,88);'
!mysql praxisDB -e 'insert into km_data (id, cluster_id,x1,x2) values (11,0,180,67);'
!mysql praxisDB -e 'insert into km_data (id, cluster_id,x1,x2) values (12,0,177,76);'

In [23]:
!mysql praxisDB -e 'call kmeans(2);'

In [24]:
!mysql praxisDB -e 'select * from km_clusters;'

+----+-------+------+
| id | x1    | x2   |
+----+-------+------+
|  1 | 181.4 | 74.5 |
|  2 |   169 |   58 |
+----+-------+------+


ROW_COUNT() return number of rows updated in the previous SQL statement <br>
see this [page](http://www.java2s.com/Tutorial/MySQL/0400__Information-Functions/ROWCOUNTreturnsthenumberofrowsupdatedinsertedordeletedbytheprecedingstatement.htm)

#K Means with Iris Dataset <br>
taken from this [site](https://github.com/antmarakis/Machine-Learning/blob/master/Clustering/kMeans%20-%20Standard/data.txt)

In [25]:
!wget https://raw.githubusercontent.com/antmarakis/Machine-Learning/master/Clustering/kMeans%20-%20Standard/data.txt

--2021-08-29 12:29:28--  https://raw.githubusercontent.com/antmarakis/Machine-Learning/master/Clustering/kMeans%20-%20Standard/data.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.110.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4698 (4.6K) [text/plain]
Saving to: ‘data.txt’

data.txt            100%[===================>]   4.59K  --.-KB/s    in 0s      

2021-08-29 12:29:28 (52.4 MB/s) - ‘data.txt’ saved [4698/4698]



In [26]:
!head data.txt

5.1,3.5,1.4,0.2,Iris-setosa
4.9,3.0,1.4,0.2,Iris-setosa
4.7,3.2,1.3,0.2,Iris-setosa
4.6,3.1,1.5,0.2,Iris-setosa
5.0,3.6,1.4,0.2,Iris-setosa
5.4,3.9,1.7,0.4,Iris-setosa
4.6,3.4,1.4,0.3,Iris-setosa
5.0,3.4,1.5,0.2,Iris-setosa
4.4,2.9,1.4,0.2,Iris-setosa
4.9,3.1,1.5,0.1,Iris-setosa


In [27]:
!mysql praxisDB -e 'drop table if exists iris_data;'
!mysql praxisDB -e 'create table iris_data (x1 float, x2 float, x3 float, x4 float, label varchar(16));'

In [28]:
!mysql praxisDB -e 'desc iris_data'

+-------+-------------+------+-----+---------+-------+
| Field | Type        | Null | Key | Default | Extra |
+-------+-------------+------+-----+---------+-------+
| x1    | float       | YES  |     | NULL    |       |
| x2    | float       | YES  |     | NULL    |       |
| x3    | float       | YES  |     | NULL    |       |
| x4    | float       | YES  |     | NULL    |       |
| label | varchar(16) | YES  |     | NULL    |       |
+-------+-------------+------+-----+---------+-------+


In [ ]:
#!mysql praxisDB -e 'truncate iris_data;'
#!mysql praxisDB -e 'select count(*) from iris_data;'

In [29]:
!mysql praxisDB -e "LOAD DATA LOCAL INFILE 'data.txt' INTO TABLE iris_data FIELDS TERMINATED BY ',' LINES TERMINATED BY '\r\n' ;"

In [30]:
!mysql praxisDB -e 'select count(*) from iris_data;'

+----------+
| count(*) |
+----------+
|      150 |
+----------+


In [31]:
!mysql praxisDB -e 'select * from iris_data limit 5;'

+------+------+------+------+-------------+
| x1   | x2   | x3   | x4   | label       |
+------+------+------+------+-------------+
|  5.1 |  3.5 |  1.4 |  0.2 | Iris-setosa |
|  4.9 |    3 |  1.4 |  0.2 | Iris-setosa |
|  4.7 |  3.2 |  1.3 |  0.2 | Iris-setosa |
|  4.6 |  3.1 |  1.5 |  0.2 | Iris-setosa |
|    5 |  3.6 |  1.4 |  0.2 | Iris-setosa |
+------+------+------+------+-------------+


In [32]:
!mysql praxisDB -e 'drop table if exists km_iris;'
!mysql praxisDB -e 'create table km_iris (id int auto_increment primary key, clusterID int, x1 float, x2 float, x3 float, x4 float, label varchar(16));'

In [33]:
!mysql praxisDB -e 'desc km_iris;'

+-----------+-------------+------+-----+---------+----------------+
| Field     | Type        | Null | Key | Default | Extra          |
+-----------+-------------+------+-----+---------+----------------+
| id        | int(11)     | NO   | PRI | NULL    | auto_increment |
| clusterID | int(11)     | YES  |     | NULL    |                |
| x1        | float       | YES  |     | NULL    |                |
| x2        | float       | YES  |     | NULL    |                |
| x3        | float       | YES  |     | NULL    |                |
| x4        | float       | YES  |     | NULL    |                |
| label     | varchar(16) | YES  |     | NULL    |                |
+-----------+-------------+------+-----+---------+----------------+


In [34]:
%%writefile step10.sql
INSERT INTO km_iris (clusterID,x1,x2,x3,x4,label)
SELECT '0',x1,x2,x3,x4,label
FROM iris_data;

Writing step10.sql


In [35]:
!mysql praxisDB --table < step10.sql

In [36]:
!mysql praxisDB -e 'select * from km_iris limit 10;'

+----+-----------+------+------+------+------+-------------+
| id | clusterID | x1   | x2   | x3   | x4   | label       |
+----+-----------+------+------+------+------+-------------+
|  1 |         0 |  5.1 |  3.5 |  1.4 |  0.2 | Iris-setosa |
|  2 |         0 |  4.9 |    3 |  1.4 |  0.2 | Iris-setosa |
|  3 |         0 |  4.7 |  3.2 |  1.3 |  0.2 | Iris-setosa |
|  4 |         0 |  4.6 |  3.1 |  1.5 |  0.2 | Iris-setosa |
|  5 |         0 |    5 |  3.6 |  1.4 |  0.2 | Iris-setosa |
|  6 |         0 |  5.4 |  3.9 |  1.7 |  0.4 | Iris-setosa |
|  7 |         0 |  4.6 |  3.4 |  1.4 |  0.3 | Iris-setosa |
|  8 |         0 |    5 |  3.4 |  1.5 |  0.2 | Iris-setosa |
|  9 |         0 |  4.4 |  2.9 |  1.4 |  0.2 | Iris-setosa |
| 10 |         0 |  4.9 |  3.1 |  1.5 |  0.1 | Iris-setosa |
+----+-----------+------+------+------+------+-------------+


In [37]:
%%writefile step11.sql

drop table if exists km_clusters;  -- contains the final clusters centres
drop table if exists km_steps;     -- shows intermediate steps
create table km_clusters (id int auto_increment primary key, x1 float, x2 float, x3 float, x4 float);
create table km_steps (id int, x1 float, x2 float, x3 float, x4 float);

Writing step11.sql


In [38]:
!mysql praxisDB --table < step11.sql

In [41]:
%%writefile step14.sql
drop procedure if exists kmeans;
DELIMITER //
CREATE PROCEDURE kmeans(v_K int)
BEGIN
TRUNCATE km_clusters;
truncate km_steps;

-- initialize cluster centers
INSERT INTO km_clusters (x1, x2, x3,x4) SELECT x1, x2,x3,x4 FROM km_iris LIMIT v_K;
REPEAT
    insert into km_steps select * from km_clusters;
    -- assign clusters to data points
    UPDATE km_iris d SET clusterID = (SELECT id FROM km_clusters c 
        ORDER BY POW(d.x1-c.x1,2)+POW(d.x2-c.x2,2)+POW(d.x3-c.x3,2)+POW(d.x4-c.x4,2) ASC LIMIT 1);
    -- calculate new cluster center
    UPDATE km_clusters C, (SELECT clusterID, 
        AVG(x1) AS x1, AVG(x2) AS x2, AVG(x3) AS x3, AVG(x4) AS x4 
        FROM km_iris GROUP BY clusterID) D 
    SET C.x1=D.x1, C.x2=D.x2, C.x3=D.x3, C.x4=D.x4 WHERE C.id=D.clusterID;
UNTIL ROW_COUNT() = 0 END REPEAT;
END//

Overwriting step14.sql


In [42]:
!mysql praxisDB --table < step14.sql

In [43]:
!mysql praxisDB -e 'select * from km_clusters;'

In [44]:
!mysql praxisDB -e 'select * from km_steps;'

In [45]:
!mysql praxisDB -e 'call kmeans(3)'

In [46]:
# look at 10 random rows from the output after clusters have been identified
!mysql praxisDB -e 'select * from km_iris order by rand() limit 10;'

+-----+-----------+------+------+------+------+-----------------+
| id  | clusterID | x1   | x2   | x3   | x4   | label           |
+-----+-----------+------+------+------+------+-----------------+
|  28 |         3 |  5.2 |  3.5 |  1.5 |  0.2 | Iris-setosa     |
|  85 |         2 |  5.4 |    3 |  4.5 |  1.5 | Iris-versicolor |
| 125 |         1 |  6.7 |  3.3 |  5.7 |  2.1 | Iris-virginica  |
| 126 |         1 |  7.2 |  3.2 |    6 |  1.8 | Iris-virginica  |
| 137 |         1 |  6.3 |  3.4 |  5.6 |  2.4 | Iris-virginica  |
| 136 |         1 |  7.7 |    3 |  6.1 |  2.3 | Iris-virginica  |
| 114 |         2 |  5.7 |  2.5 |    5 |    2 | Iris-virginica  |
| 146 |         1 |  6.7 |    3 |  5.2 |  2.3 | Iris-virginica  |
|  52 |         2 |  6.4 |  3.2 |  4.5 |  1.5 | Iris-versicolor |
|  49 |         3 |  5.3 |  3.7 |  1.5 |  0.2 | Iris-setosa     |
+-----+-----------+------+------+------+------+-----------------+


In [47]:
!mysql praxisDB -e 'select * from km_steps;'

+------+---------+---------+---------+---------+
| id   | x1      | x2      | x3      | x4      |
+------+---------+---------+---------+---------+
|    1 |     5.1 |     3.5 |     1.4 |     0.2 |
|    2 |     4.9 |       3 |     1.4 |     0.2 |
|    3 |     4.7 |     3.2 |     1.3 |     0.2 |
|    1 |  6.1618 | 3.25955 | 4.02809 | 1.33258 |
|    2 |  5.5451 | 2.67059 | 3.76471 | 1.16078 |
|    3 |    4.53 |    3.18 |    1.33 |     0.2 |
|    1 | 6.53014 | 2.98356 | 5.24521 | 1.84247 |
|    2 | 5.53704 | 2.57037 | 3.98889 | 1.22593 |
|    3 |   5.006 |   3.418 |   1.464 |   0.244 |
|    1 | 6.57727 | 2.99091 | 5.34848 | 1.89545 |
|    2 |    5.65 | 2.64118 | 4.04706 |    1.25 |
|    3 |   5.006 |   3.418 |   1.464 |   0.244 |
|    1 | 6.60159 | 2.98571 | 5.38413 | 1.91587 |
|    2 | 5.68378 | 2.67838 | 4.09189 | 1.26757 |
|    3 |   5.006 |   3.418 |   1.464 |   0.244 |
|    1 |  6.6322 | 2.99831 | 5.43051 | 1.93729 |
|    2 | 5.72927 | 2.69024 | 4.15122 |     1.3 |
|    3 |   5.006 |  

# Predictions & Labels

In [48]:
!mysql praxisDB -e 'select clusterID, label from km_iris;'

+-----------+-----------------+
| clusterID | label           |
+-----------+-----------------+
|         3 | Iris-setosa     |
|         3 | Iris-setosa     |
|         3 | Iris-setosa     |
|         3 | Iris-setosa     |
|         3 | Iris-setosa     |
|         3 | Iris-setosa     |
|         3 | Iris-setosa     |
|         3 | Iris-setosa     |
|         3 | Iris-setosa     |
|         3 | Iris-setosa     |
|         3 | Iris-setosa     |
|         3 | Iris-setosa     |
|         3 | Iris-setosa     |
|         3 | Iris-setosa     |
|         3 | Iris-setosa     |
|         3 | Iris-setosa     |
|         3 | Iris-setosa     |
|         3 | Iris-setosa     |
|         3 | Iris-setosa     |
|         3 | Iris-setosa     |
|         3 | Iris-setosa     |
|         3 | Iris-setosa     |
|         3 | Iris-setosa     |
|         3 | Iris-setosa     |
|         3 | Iris-setosa     |
|         3 | Iris-setosa     |
|         3 | Iris-setosa     |
|         3 | Iris-setosa     |
|       

In [49]:
%%writefile step21.sql
select id, label,  
case
  when label = 'Iris-setosa' then 3
  when label = 'Iris-versicolor' then 2
  when label = 'Iris-virginica' then 1
  else  0
end
labelID,
clusterID
from km_iris

Writing step21.sql


In [50]:
!mysql praxisDB --table < step21.sql

+-----+-----------------+---------+-----------+
| id  | label           | labelID | clusterID |
+-----+-----------------+---------+-----------+
|   1 | Iris-setosa     |       3 |         3 |
|   2 | Iris-setosa     |       3 |         3 |
|   3 | Iris-setosa     |       3 |         3 |
|   4 | Iris-setosa     |       3 |         3 |
|   5 | Iris-setosa     |       3 |         3 |
|   6 | Iris-setosa     |       3 |         3 |
|   7 | Iris-setosa     |       3 |         3 |
|   8 | Iris-setosa     |       3 |         3 |
|   9 | Iris-setosa     |       3 |         3 |
|  10 | Iris-setosa     |       3 |         3 |
|  11 | Iris-setosa     |       3 |         3 |
|  12 | Iris-setosa     |       3 |         3 |
|  13 | Iris-setosa     |       3 |         3 |
|  14 | Iris-setosa     |       3 |         3 |
|  15 | Iris-setosa     |       3 |         3 |
|  16 | Iris-setosa     |       3 |         3 |
|  17 | Iris-setosa     |       3 |         3 |
|  18 | Iris-setosa     |       3 |     

In [52]:
%%writefile step22.sql

select K2.id, if(K2.labelID = K2.clusterID,'match','error') checkPL from (
  select id, label,  
    case
    when label = 'Iris-setosa' then 3
    when label = 'Iris-versicolor' then 2
    when label = 'Iris-virginica' then 1
    else  0
    end
  labelID,
  clusterID
  from km_iris
  ) K2

Overwriting step22.sql


In [53]:
!mysql praxisDB --table < step22.sql

+-----+---------+
| id  | checkPL |
+-----+---------+
|   1 | match   |
|   2 | match   |
|   3 | match   |
|   4 | match   |
|   5 | match   |
|   6 | match   |
|   7 | match   |
|   8 | match   |
|   9 | match   |
|  10 | match   |
|  11 | match   |
|  12 | match   |
|  13 | match   |
|  14 | match   |
|  15 | match   |
|  16 | match   |
|  17 | match   |
|  18 | match   |
|  19 | match   |
|  20 | match   |
|  21 | match   |
|  22 | match   |
|  23 | match   |
|  24 | match   |
|  25 | match   |
|  26 | match   |
|  27 | match   |
|  28 | match   |
|  29 | match   |
|  30 | match   |
|  31 | match   |
|  32 | match   |
|  33 | match   |
|  34 | match   |
|  35 | match   |
|  36 | match   |
|  37 | match   |
|  38 | match   |
|  39 | match   |
|  40 | match   |
|  41 | match   |
|  42 | match   |
|  43 | match   |
|  44 | match   |
|  45 | match   |
|  46 | match   |
|  47 | match   |
|  48 | match   |
|  49 | match   |
|  50 | match   |
|  51 | error   |
|  52 | match   |
|  53 | er

In [54]:
%%writefile step23.sql

select 
count(if(K3.checkPL = 'match',1,NULL)) km_match,
count(if(K3.checkPL = 'error',1,NULL)) km_error, 
count(if(K3.checkPL = 'match',1,NULL)) / (count(if(K3.checkPL = 'match',1,NULL)) +count(if(K3.checkPL = 'error',1,NULL))) success
from (
  select K2.id, if(K2.labelID = K2.clusterID,'match','error') checkPL from (
    select id, label,  
      case
      when label = 'Iris-setosa' then 3
      when label = 'Iris-versicolor' then 2
      when label = 'Iris-virginica' then 1
      else  0
      end
     labelID,
     clusterID
    from km_iris
    ) K2
  ) K3

Writing step23.sql


In [55]:
!mysql praxisDB --table < step23.sql

+----------+----------+---------+
| km_match | km_error | success |
+----------+----------+---------+
|      133 |       17 |  0.8867 |
+----------+----------+---------+


#Chronobooks <br>
![alt text](https://1.bp.blogspot.com/-lTiYBkU2qbU/X1er__fvnkI/AAAAAAAAjtE/GhDR3OEGJr4NG43fZPodrQD5kbxtnKebgCLcBGAsYHQ/s600/Footer2020-600x200.png)<hr>
Chronotantra and Chronoyantra are two science fiction novels that explore the collapse of human civilisation on Earth and then its rebirth and reincarnation both on Earth as well as on the distant worlds of Mars, Titan and Enceladus. But is it the human civilisation that is being reborn? Or is it some other sentience that is revealing itself. 
If you have an interest in AI and found this material useful, you may consider buying these novel, in paperback or kindle, from [http://bit.ly/chronobooks](http://bit.ly/chronobooks)